In [1]:
import gym, time, math, copy, ray, random
import numpy as np
import matplotlib.pyplot as plt
ray.shutdown()
ray.init()
time.sleep(2)

ImportError: No module named 'gym'

In [2]:
from gym.wrappers import Monitor

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class gameAI(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc = nn.Sequential(
                        nn.Linear(game_observation ,128, bias=True),
                        nn.ReLU(),
                        nn.Linear(128, game_actions, bias=True),
                        nn.Tanh()
                        )

                
        def forward(self, inputs):
            x = self.fc(inputs)
            return x

In [5]:
def init_weights(m):
    
        # nn.Conv2d weights are of shape [16, 1, 3, 3] i.e. # number of filters, 1, stride, stride
        # nn.Conv2d bias is of shape [16] i.e. # number of filters
        
        # nn.Linear weights are of shape [32, 24336] i.e. # number of input features, number of output features
        # nn.Linear bias is of shape [32] i.e. # number of output features
        
        if ((type(m) == nn.Linear) | (type(m) == nn.Conv2d)):
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.00)
                

In [6]:
def return_random_agents(num_agents):
    
    agents = []
    for _ in range(num_agents):
        
        agent = gameAI()
        
        for param in agent.parameters():
            param.requires_grad = False
        init_weights(agent)
        
        agents.append(agent)
    return agents
# agents = return_random_agents(250)

In [7]:
m = torch.nn.Tanh()
m(torch.tensor([2.0]))

tensor([0.9640])

In [8]:
@ray.remote
def run1(agents, env, display = False):
    reward_agents = []
    for agent in agents:
        agent.eval() # I think to set it to evaluation mode and not to training mode
        observation = env.reset()
        r=0
        for _ in range(250):
            if(display):
                env.render()
            observation = torch.tensor(observation)
#             for i in range(len(observation)):
#                 if(env.observation_space.high[i] == float('Inf')):
#                     pass
#                 else:
#                     observation[i]*=env.observation_space.high[i]
#             print(observation)
            inp = observation.type('torch.FloatTensor').view(1,-1) 
            # tensor([[-0.0033, -0.0465, -0.0114, -0.0097]])
            
            action = agent(inp).detach().numpy()[0]
            for i in range(len(action)):
                action[i]*=env.action_space.high[i]
#             print(action)
            
            observation, reward, done, info = env.step(action)
            r=r+reward
            if(done):
                break
        reward_agents.append(r)        
        #reward_agents.append(s)
    if(display):
        env.close()
    return reward_agents

# run1([agents[0]], gym.make(game))

In [9]:
# %%time
# this function divides agents into {no_of_cores sets of agents} 
def run_agents(agents):
    envS = []
    for i in range(no_of_cores):
        envS.append(gym.make(game))
    
    agents = np.array(agents)
    agents = agents.reshape(no_of_cores, -1)

    result_ids = []
    for i in range(no_of_cores):
        result_ids.append(run1.remote(agents[i], envS[i]))

    results = ray.get(result_ids)
    results = np.array(results, dtype=int)
    return results.reshape(agents.shape[0]*agents.shape[1], -1)

In [10]:
def run_agents_n_times(agents, runs):
    avg_score = np.zeros((len(agents),1))
    for i in range(runs):
        avg_score += run_agents(agents)
    avg_score/=runs
    avg_score = avg_score.reshape(len(agents))
    return avg_score
# %time run_agents_n_times(agents, 1)

In [11]:
def add_elite(agents, sorted_parent_indexes, elite_index=None):
    only_consider_top_n = no_of_cores*2
    print('Only considering top ',only_consider_top_n, ' for elite selection.')
    candidate_elite_index = sorted_parent_indexes[:only_consider_top_n]
    if(elite_index is not None):
        candidate_elite_index = np.append([elite_index], candidate_elite_index)
    candidate_elite_index = candidate_elite_index[:only_consider_top_n]
    # [74, 105, 52, 278, 892, .., 645] 
    
    candidate_elite_agents = []
    for i in candidate_elite_index:
        candidate_elite_agents.append(agents[i])
    candidate_elite_agents = np.array(candidate_elite_agents)
    times = 5
    rewards = run_agents_n_times(candidate_elite_agents, times)
    # [score, score, score, .., score]
    print('Running each elite candidate ', times, ' times.')
    
    top_score = None
    top_elite_index = None
    
    for i in range(len(rewards)):
        score = rewards[i]
        print("Score for elite i ", candidate_elite_index[i], " is ", score)
        
        if(top_score is None):
            top_score = score
            top_elite_index = candidate_elite_index[i]
        elif(score >= top_score):
            top_score = score
            top_elite_index = candidate_elite_index[i]
            
    print("Elite selected with index ",top_elite_index, " and score", top_score)
    
    child_agent = copy.deepcopy(agents[top_elite_index])
    return child_agent
    

In [12]:
def mutate_each_param(param):
    param += np.random.randn()
    return param
@ray.remote
def mutate(agents):
    child_agents = []
    for agent in agents:
        child_agent = copy.deepcopy(agent)
        for param in child_agent.parameters():
    #         print(param.shape)
            if(len(param.shape)==2): #weights of linear layer
                total = param.shape[0]*param.shape[1]
                to_mutate = random.sample(range(total), round(total*mutation_rate))
    #             print(len(to_mutate))
                for i in to_mutate:
                    param[i%param.shape[0]][int(i/param.shape[0])] = mutate_each_param(param[i%param.shape[0]][int(i/param.shape[0])])
            elif(len(param.shape)==1): #biases of linear layer or conv layer
                to_mutate = random.sample(range(param.shape[0]), max(round(param.shape[0]*mutation_rate), random.randint(0,1)) )
    #             print(to_mutate)
                for i in to_mutate:
                    param[i] = mutate_each_param(param[i])
        child_agents.append(child_agent)
    return child_agents
def mutate_all(agents):    
    agents = np.array(agents)
    agents = agents.reshape(no_of_cores, -1)
    result_ids = []
    for i in range(no_of_cores):
        result_ids.append(mutate.remote(agents[i]))
    child_agents = ray.get(result_ids)
    child_agents = np.array(child_agents)
    return child_agents.reshape(agents.shape[0]*agents.shape[1])
    
# %time mutate_all(agents)
# %time [mutate(agent) for agent in agents]
# print('')

In [13]:
def return_children(agents, sorted_parent_indexes, elite_index):
    
    children_agents = []
    
    #first take selected parents from sorted_parent_indexes and generate N-1 children
    for i in range(len(agents)):
        
        selected_agent_index = sorted_parent_indexes[np.random.randint(len(sorted_parent_indexes))]
        children_agents.append(agents[selected_agent_index]) #add mutate here
    children_agents = mutate_all(children_agents)

    #now add one elite
    children_agents = children_agents.tolist()
    children_agents.pop(0)
    elite_child = add_elite(agents, sorted_parent_indexes, elite_index)
    children_agents.append(elite_child)
    elite_index=len(children_agents)-1 #it is the last one
    
    return children_agents, elite_index

# %time return_children(agents, sorted_parent_indexes, elite_index)

In [14]:
game = 'BipedalWalker-v2'
game_observation = 24
game_actions = 4

env = gym.make(game)
print(env.observation_space)
print(env.action_space)
print(env.observation_space.low, env.observation_space.high)
print(env.action_space.low, env.action_space.high)
print(env.action_space.sample())

#disable gradients as we will not use them
torch.set_grad_enabled(False)

no_of_cores = 8
# initialize N number of agents
num_agents = 600
agents = return_random_agents(num_agents)

# How many top agents to consider as parents
top_limit = 20

mutation_rate = 0.05
#     mutation_power = 0.02 #hyper-parameter, set from https://arxiv.org/pdf/1712.06567.pdf  

# run evolution until X generations
generations = 50

elite_index = None
for generation in range(generations):
    # return rewards of agents
    rewards = run_agents_n_times(agents, 3) #return average of 3 runs

    # sort by rewards
    sorted_parent_indexes = np.argsort(rewards)[::-1][:top_limit] #reverses and gives top values (argsort sorts by ascending by default) https://stackoverflow.com/questions/16486252/is-it-possible-to-use-argsort-in-descending-order
    # [ 92 785 321 682 342  27 946 464  41  21 867 774 893 431 628 399 997 708 820 739]
    
    top_rewards = []
    for best_parent in sorted_parent_indexes:
        top_rewards.append(rewards[best_parent])
    
    print("Generation ", generation, " | Mean rewards: ", int(np.mean(rewards)), " | Mean of top 5: ",int(np.mean(top_rewards[:5])), " | Mean of top ",top_limit," : ", int(np.mean(top_rewards[:top_limit])))
    print("Top ",top_limit," scores", sorted_parent_indexes)
    print("Rewards for top: ",np.array(top_rewards).astype(int))
    
    # setup an empty list for containing children agents
    children_agents, elite_index = return_children(agents, sorted_parent_indexes, elite_index)

    # kill all agents, and replace them with their children
    agents = children_agents
    
    print("------------------------------------------------------------------------------")
    print('')

Box(24,)
Box(4,)
[-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf] [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf]
[-1. -1. -1. -1.] [1. 1. 1. 1.]
[0.7743805  0.58140355 0.4466682  0.06240915]


2019-07-04 14:47:17,203	WARNING worker.py:2041 -- WARNING: Could not produce a deterministic class ID for class <class '__main__.gameAI'>
2019-07-04 14:47:17,210	WARNING worker.py:342 -- WARNING: Falling back to serializing objects of type <class 'torch.nn.backends.thnn.THNNFunctionBackend'> by using pickle. This may be inefficient.
2019-07-04 14:47:17,329	WARNING worker.py:342 -- WARNING: Falling back to serializing objects of type <class 'numpy.dtype'> by using pickle. This may be inefficient.
2019-07-04 14:47:17,336	WARNING worker.py:342 -- WARNING: Falling back to serializing objects of type <class 'mtrand.RandomState'> by using pickle. This may be inefficient.
2019-07-04 14:47:17,363	WARNING worker.py:353 -- WARNING: Pickling the class <class 'SwigPyObject'> failed, so we are using pickle and only registering the class locally.
2019-07-04 14:47:17,377	WARNING worker.py:402 -- WARNING: Serializing the class <class 'gym.wrappers.time_limit.TimeLimit'> failed, so are are falling back

Generation  0  | Mean rewards:  -69  | Mean of top 5:  5  | Mean of top  20  :  3
Top  20  scores [491  52 200  16 151 430  63 580  38 143 356 490 399 390   0  31   2 442
 576 184]
Rewards for top:  [6 6 5 5 4 4 4 3 3 3 3 3 3 3 3 2 2 2 2 2]
Only considering top  16  for elite selection.
Running each elite candidate  5  times.
Score for elite i  491  is  5.6
Score for elite i  52  is  4.0
Score for elite i  200  is  -93.4
Score for elite i  16  is  -19.2
Score for elite i  151  is  4.0
Score for elite i  430  is  4.0
Score for elite i  63  is  3.4
Score for elite i  580  is  5.0
Score for elite i  38  is  3.2
Score for elite i  143  is  3.2
Score for elite i  356  is  3.2
Score for elite i  490  is  3.2
Score for elite i  399  is  3.2
Score for elite i  390  is  2.2
Score for elite i  0  is  4.6
Score for elite i  31  is  2.4
Elite selected with index  491  and score 5.6
------------------------------------------------------------------------------

Generation  1  | Mean rewards:  -75  

Only considering top  16  for elite selection.
Running each elite candidate  5  times.
Score for elite i  599  is  5.8
Score for elite i  599  is  5.4
Score for elite i  435  is  -22.4
Score for elite i  310  is  -9.6
Score for elite i  219  is  -33.2
Score for elite i  354  is  -10.2
Score for elite i  174  is  -7.8
Score for elite i  517  is  -77.8
Score for elite i  353  is  -31.4
Score for elite i  247  is  -10.2
Score for elite i  221  is  -10.2
Score for elite i  211  is  -11.6
Score for elite i  113  is  -12.0
Score for elite i  177  is  -103.6
Score for elite i  519  is  -11.0
Score for elite i  140  is  -76.2
Elite selected with index  599  and score 5.8
------------------------------------------------------------------------------

Generation  9  | Mean rewards:  -65  | Mean of top 5:  -2  | Mean of top  20  :  -8
Top  20  scores [599 423 406 238 399 235 126 552 566 224 140 255 513  68 505 261 297 111
 183 476]
Rewards for top:  [  5   0  -4  -7  -7  -7  -7  -8  -9  -9  -9  -

Generation  17  | Mean rewards:  -51  | Mean of top 5:  21  | Mean of top  20  :  15
Top  20  scores [371 350 296 135 536 410 208 431 238  46 181 571 154 245 543 132 260 366
 137 321]
Rewards for top:  [28 23 22 17 16 15 15 15 15 14 14 14 13 13 11 11 11 10 10 10]
Only considering top  16  for elite selection.
Running each elite candidate  5  times.
Score for elite i  599  is  -8.4
Score for elite i  371  is  -18.8
Score for elite i  350  is  17.6
Score for elite i  296  is  12.6
Score for elite i  135  is  11.6
Score for elite i  536  is  16.0
Score for elite i  410  is  -22.0
Score for elite i  208  is  11.6
Score for elite i  431  is  -22.8
Score for elite i  238  is  11.8
Score for elite i  46  is  6.8
Score for elite i  181  is  15.0
Score for elite i  571  is  0.6
Score for elite i  154  is  9.2
Score for elite i  245  is  3.2
Score for elite i  543  is  -98.4
Elite selected with index  350  and score 17.6
---------------------------------------------------------------------------

Only considering top  16  for elite selection.
Running each elite candidate  5  times.
Score for elite i  599  is  35.0
Score for elite i  50  is  -9.4
Score for elite i  241  is  38.4
Score for elite i  142  is  8.2
Score for elite i  599  is  38.6
Score for elite i  174  is  25.6
Score for elite i  179  is  6.0
Score for elite i  369  is  -26.0
Score for elite i  304  is  22.4
Score for elite i  99  is  5.0
Score for elite i  287  is  26.8
Score for elite i  521  is  29.6
Score for elite i  431  is  -50.0
Score for elite i  403  is  35.2
Score for elite i  185  is  27.4
Score for elite i  130  is  -53.8
Elite selected with index  599  and score 38.6
------------------------------------------------------------------------------

Generation  26  | Mean rewards:  -44  | Mean of top 5:  39  | Mean of top  20  :  34
Top  20  scores [ 65 378 114  70 359 115 126 173 597 350 283 116 280 524 599  15 500 408
 456  72]
Rewards for top:  [45 40 37 37 36 36 36 35 34 34 33 33 33 32 32 31 31 31 30 

Generation  34  | Mean rewards:  -44  | Mean of top 5:  39  | Mean of top  20  :  35
Top  20  scores [285 377 309  81 116  89  39 532 231  43 180 104 540 140 386 492 227  75
 439  48]
Rewards for top:  [42 40 39 38 38 38 38 38 37 34 34 34 33 33 32 32 31 31 31 31]
Only considering top  16  for elite selection.
Running each elite candidate  5  times.
Score for elite i  599  is  10.8
Score for elite i  285  is  -15.6
Score for elite i  377  is  13.8
Score for elite i  309  is  36.4
Score for elite i  81  is  -39.0
Score for elite i  116  is  41.8
Score for elite i  89  is  24.0
Score for elite i  39  is  3.4
Score for elite i  532  is  12.4
Score for elite i  231  is  15.4
Score for elite i  43  is  -28.2
Score for elite i  180  is  7.2
Score for elite i  104  is  31.8
Score for elite i  540  is  9.2
Score for elite i  140  is  33.8
Score for elite i  386  is  15.0
Elite selected with index  116  and score 41.8
------------------------------------------------------------------------------

Only considering top  16  for elite selection.
Running each elite candidate  5  times.
Score for elite i  599  is  12.2
Score for elite i  323  is  -13.2
Score for elite i  416  is  -36.8
Score for elite i  193  is  19.4
Score for elite i  386  is  -13.4
Score for elite i  383  is  -39.2
Score for elite i  40  is  40.2
Score for elite i  533  is  -28.2
Score for elite i  176  is  15.8
Score for elite i  366  is  -65.8
Score for elite i  52  is  16.6
Score for elite i  361  is  -39.6
Score for elite i  81  is  -7.0
Score for elite i  99  is  11.6
Score for elite i  376  is  15.8
Score for elite i  11  is  -73.2
Elite selected with index  40  and score 40.2
------------------------------------------------------------------------------

Generation  43  | Mean rewards:  -40  | Mean of top 5:  45  | Mean of top  20  :  42
Top  20  scores [ 37 413 438 153  43 291 476 530 251 162 367 448 390  92 187 285 379 146
 241 405]
Rewards for top:  [48 46 44 44 43 43 43 43 43 42 42 42 42 41 41 41 40 40

In [25]:
import time
def play_agent(agent):
    try: #try and exception block because, render hangs if an erorr occurs, we must do env.close to continue working    
        agents = [agent]
        env = gym.make(game)
        display = True
        reward_agents = []
        for agent in agents:
            agent.eval() # I think to set it to evaluation mode and not to training mode
            observation = env.reset()
            r=0
            for _ in range(1500):
                if(display):
                    env.render()
                observation = torch.tensor(observation)
                for i in range(len(observation)):
                    if(env.observation_space.high[i] == float('Inf')):
                        pass
                    else:
                        observation[i]*=env.observation_space.high[i]
    #             print(observation)
                inp = observation.type('torch.FloatTensor').view(1,-1) 
                # tensor([[-0.0033, -0.0465, -0.0114, -0.0097]])

                action = agent(inp).detach().numpy()[0]
                for i in range(len(action)):
                    action[i]*=env.action_space.high[i]
    #             print(action)

                observation, reward, done, info = env.step(action)
                r=r+reward
                if(done):
                    print(r)
                    break
            reward_agents.append(r)        
            #reward_agents.append(s)
        if(display):
            env.close()

    except Exception as e:
        env_record.close()
        print(e.__doc__)
        print(e.message)        

In [50]:
# play_agent(agents[elite_index])
torch.save(agents[elite_index].fc, './Elite.gameAI')


In [17]:
# env = gym.make(game)
# def play_agent(agent):
#     try: #try and exception block because, render hangs if an erorr occurs, we must do env.close to continue working    
#         r = ray.get(run1.remote([agent], env, True))
#         env.close()
#         print("Rewards: ",r)

#     except Exception as e:
#         print("EXCEPTION OCCURED")
#         env.close()   

# Code for continuing training on saved model